In [24]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

def number_to_digits(n):
    return [int(d) for d in str(n)]

num_samples = 20000
# For debugging, limit the maximum number to 100
X_numbers = np.random.randint(0, 100_000, size=num_samples)

X = [number_to_digits(x) for x in X_numbers]
y = [number_to_digits(int(str(x)[::-1])) for x in X_numbers]

max_len = max(max(len(x) for x in X), max(len(x) for x in y))

for x in X:
    while len(x) < max_len:
        x.insert(0, 0)

for x in y:
    while len(x) < max_len:
        x.insert(0, 0)

X = np.array(X)
y = np.array(y)

split = int(0.8 * num_samples)
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

class BiLSTMSeq2Seq(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(BiLSTMSeq2Seq, self).__init__()

        self.embedding = nn.Embedding(input_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out)
        return output

input_dim = 10
hidden_dim = 64
output_dim = 10
num_layers = 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net = BiLSTMSeq2Seq(input_dim, hidden_dim, output_dim, num_layers).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0005)

X_train_tensor = torch.LongTensor(X_train).to(device)
y_train_tensor = torch.LongTensor(y_train).to(device)
X_val_tensor = torch.LongTensor(X_val).to(device)
y_val_tensor = torch.LongTensor(y_val).to(device)

epochs = 3500
for epoch in range(epochs):
    net.train()
    optimizer.zero_grad()
    outputs = net(X_train_tensor)

    outputs_reshaped = outputs.view(-1, 10)
    y_train_reshaped = y_train_tensor.view(-1)

    loss = criterion(outputs_reshaped, y_train_reshaped)
    loss.backward()
    optimizer.step()

    net.eval()
    with torch.no_grad():
        val_outputs = net(X_val_tensor)
        val_outputs_reshaped = val_outputs.view(-1, 10)
        y_val_reshaped = y_val_tensor.view(-1)
        val_loss = criterion(val_outputs_reshaped, y_val_reshaped)

    if epoch % 100 == 0:
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {loss.item()} - Val Loss: {val_loss.item()}")


Epoch 1/3500 - Loss: 2.3004753589630127 - Val Loss: 2.297133684158325
Epoch 101/3500 - Loss: 1.1983234882354736 - Val Loss: 1.197767734527588
Epoch 201/3500 - Loss: 0.7542758584022522 - Val Loss: 0.7582878470420837
Epoch 301/3500 - Loss: 0.49173837900161743 - Val Loss: 0.49660712480545044
Epoch 401/3500 - Loss: 0.3008958399295807 - Val Loss: 0.3053213655948639
Epoch 501/3500 - Loss: 0.1747964471578598 - Val Loss: 0.17900510132312775
Epoch 601/3500 - Loss: 0.10232695192098618 - Val Loss: 0.10600744932889938
Epoch 701/3500 - Loss: 0.06253274530172348 - Val Loss: 0.06578018516302109
Epoch 801/3500 - Loss: 0.04036607965826988 - Val Loss: 0.04315260052680969
Epoch 901/3500 - Loss: 0.027600860223174095 - Val Loss: 0.02992376685142517
Epoch 1001/3500 - Loss: 0.019788024947047234 - Val Loss: 0.021717557683587074
Epoch 1101/3500 - Loss: 0.014762870967388153 - Val Loss: 0.016375942155718803
Epoch 1201/3500 - Loss: 0.01138725969940424 - Val Loss: 0.012749485671520233
Epoch 1301/3500 - Loss: 0.009

In [25]:
def reverse_number_prediction(model, number):
    # Convert the number to its digit sequence
    digits = number_to_digits(number)

    # Pad the sequence to max_len
    while len(digits) < max_len:
        digits.insert(0, 0)

    # Convert to tensor and move to the device
    tensor_input = torch.LongTensor([digits]).to(device)

    # Get the prediction
    model.eval()
    with torch.no_grad():
        outputs = model(tensor_input)
        _, predicted = torch.max(outputs, 2)  # Argmax over the last dimension to get predicted digit

    reversed_digits = predicted[0].cpu().numpy().tolist()

    # Convert digit sequence back to a number
    reversed_number = int(''.join(map(str, reversed_digits)))

    return reversed_number



In [30]:
# Test the function
given_number = 123887
predicted_reversed = reverse_number_prediction(net, given_number)
print(f"Given Number: {given_number}")
print(f"Predicted Reversed Number: {predicted_reversed}")


Given Number: 123887
Predicted Reversed Number: 783211
